# Instructions to run the flask API: 

- Ensure you are in the "library" folder

- Run these commands:
 ```
 >>> export FLASK_APP=gcmgenerator
 >>> flask run -h 0.0.0.0 -p 6000
 ```
- (change port and host details as required)


In [1]:
from flask import Blueprint, render_template, request, jsonify, flash
import requests
import json
from tqdm import tqdm
import time
import pandas as pd
import sys, os

## ALIGNER

- Sentences are passed here, and the alignment is generated.

In [10]:
# alignment generation
l1 = "यदि आप तुरंत डॉक्टर से संपर्क करें"
l2 = "contact the doctor immediately if you"
td = {'l1':l1, 'l2':l2}
alignment_api_endpoint = "http://127.0.0.1:6000/statistical_aligner_enhi"

response = requests.post(alignment_api_endpoint, json = td)

print(response)

print(response.json())

aligner_output = response.json()
alignments = aligner_output['alignment']

<Response [200]>
{'alignment': '0-4 1-5 2-3 3-2 4-0 5-0', 'l1': 'यदि आप तुरंत डॉक्टर से संपर्क करें', 'l2': 'contact the doctor immediately if you'}


## CODE-MIXED SENTENCE GENERATOR

- Using the given sentences and alignment, codemixed sentences are generated

### Expected Outputs

- In case of any error during code-mix sentence generation, the program errors out with the message: 
```
fail
```

- Sometimes it is possible that no alignments can be generated, in which case the program returns an empty array.
- If any alignment error occurs then it is possible for the code-mixed sentence to skip a few words as well


In [11]:
# cm-sentences generation
choice = 2  #choice for language to generate parse trees
data = {
    "lang1": l1,
    "lang2": l2,
    "alignments": alignments,
    "choice": choice
}

gcm_api_endpoint = "http://127.0.0.1:6000/gcm_enhi"
#CODE FOUND IN gcmgenerator.py

response = requests.post(gcm_api_endpoint, json=data)
print(response)
#print(response.json())

retdata = response.json()
print("Sentence 1: ", retdata['lang1'])
print("Sentence 2: ", retdata['lang2'])
print("Alignments: ", retdata['alignments'])
for i in retdata['cm_sentences']:
    print(i)

<Response [200]>
Sentence 1:  यदि आप तुरंत डॉक्टर से संपर्क करें
Sentence 2:  contact the doctor immediately if you
Alignments:  ['0-4 1-5 2-3 3-2 4-0 5-0']
[IDX]	0

[L1]	यदि आप तुरंत डॉक्टर से संपर्क करें

[L2]	contact the doctor immediately if you

[L2_Tree]	(ROOT (S (VP (VB contact) (NP (DT the) (NN doctor)) (ADVP (RB immediately)) (SBAR (IN if) (NP (PRP you))))))

Alignments	0-4 1-5 2-3 3-2 4-0 5-0

Theory	ec

[CM]contact the तुरंत if you

[TREE](ROOT (VP_e (VB_e contact) (NP_e (DT_e the)) (ADVP (RB_h तुरंत)) (SBAR (IN_e if) (NP (PRP_e you)))))



[IDX]	0

[L1]	यदि आप तुरंत डॉक्टर से संपर्क करें

[L2]	contact the doctor immediately if you

[L2_Tree]	(ROOT (S (VP (VB contact) (NP (DT the) (NN doctor)) (ADVP (RB immediately)) (SBAR (IN if) (NP (PRP you))))))

Alignments	0-4 1-5 2-3 3-2 4-0 5-0

Theory	ec

[CM]contact the तुरंत if आप

[TREE](ROOT (VP_e (VB_e contact) (NP_e (DT_e the)) (ADVP (RB_h तुरंत)) (SBAR (IN_e if) (NP (PRP_h आप)))))



[IDX]	0

[L1]	यदि आप तुरंत डॉक्टर से संपर्क